### Author: Sebastian Brubaker

In [11]:
import pandas as pd
import geopandas as gpd
import folium
import os
import re
import numpy as np
import zipfile
import shutil
import logging
import search_utilities as su
import normalization_utilities as nu
from sentence_transformers import SentenceTransformer, util
import fitz
from PIL import Image
from rapidfuzz import fuzz
from logger import setup_logger

In [5]:
# Define globals

# LOGGER = setup_logger("logger") Not the main

IMG_MODEL = SentenceTransformer("clip-ViT-B-32")
TXT_MODEL = SentenceTransformer("all-MiniLM-L6-v2")

dir_tree_df = pd.read_csv(r"experimental/eao_prj_drive_crawl_spatial.csv")
dir_tree_folders_df = dir_tree_df[dir_tree_df["is_folder"] == True]

dir_tree_2002_df = dir_tree_df[dir_tree_df["act_year"] == 2002]
dir_tree_2002_folders_df = dir_tree_2002_df[dir_tree_2002_df["is_folder"] == True]

dir_tree_2018_df = dir_tree_df[dir_tree_df["act_year"] == 2018]
dir_tree_2018_folders_df = dir_tree_2018_df[dir_tree_2018_df["is_folder"] == True]

dir_tree_spatial_df = dir_tree_df[dir_tree_df["act_year"] == 'spatial']
dir_tree_spatial_folders_df = dir_tree_spatial_df[dir_tree_spatial_df["is_folder"] == True]

print(dir_tree_df.shape)
print(dir_tree_folders_df.shape)
print(dir_tree_2002_df.shape)
print(dir_tree_2002_folders_df.shape)
print(dir_tree_2018_df.shape)
print(dir_tree_2018_folders_df.shape)
print(dir_tree_spatial_df.shape)
print(dir_tree_spatial_folders_df.shape)

(538630, 5)
(85841, 5)
(280811, 5)
(49342, 5)
(243477, 5)
(34711, 5)
(5732, 5)
(398, 5)


C:\Users\SBRUBAKE\AppData\Local\Temp\ipykernel_16852\2352343678.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dir_tree_df = pd.read_csv(r"experimental/eao_prj_drive_crawl_spatial.csv")


In [7]:
uid = 200
project_name = 'project name'
clean_name = project_name.replace(" ", "_").replace("/", "-").strip() + f"_{uid}"
clean_name

'project_name_200'

In [8]:
# Make a data structure to iterate through for batch processing project

projects = {
            'project_name_200':\
                (200, ['path/to/drive_location1', 'path/to/drive_location2', ...]),
            }
projects

{'project_name_200': (200,
  ['path/to/drive_location1', 'path/to/drive_location2', Ellipsis])}

In [ ]:
query = project_name
search_df = dir_tree_df
threshold = 10
search_field = "item_name"

# Compute fuzzy score for each item
search_df["fuzzy_score"] = search_df[search_field].apply(lambda x: fuzz.token_set_ratio(x, query))
# Filter by threshold and sort by score descending
fuzzy_results_df = search_df[search_df["fuzzy_score"] >= threshold].sort_values("fuzzy_score", ascending=False)
fuzzy_results_df.head(50)

# fuzzy_results_df.full_path.iloc[1] # Index the results